# Grammar rules generalization `2018-11-26`

Updated [Grammar-Rules-Generalization-2018-11-23_.html](http://langlearn.singularitynet.io/data/clustering_2018/html/Grammar-Rules-Generalization-2018-11-23_.html) -- improved cells 16, 18.  
Link Grammar 5.4.4, `test_grammar` updated 2018-10-19. generalization updated 2018-11-26.  
This notebook is shared as static [Grammar-Rules-Generalization-2018-11-26.html](http://langlearn.singularitynet.io/data/clustering_2018/html/Grammar-Rules-Generalization-2018-11-26.html).  
Data -- [clustering_2018/Grammar-Rules-Generalization-2018-11-26](http://langlearn.singularitynet.io/data/clustering_2018/Grammar-Rules-Generalization-2018-11-26) folder.

## Basic settings

In [1]:
import os, sys, time
from collections import OrderedDict, Counter
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
from src.grammar_learner.utl import UTC
from src.grammar_learner.read_files import check_dir
from src.grammar_learner.write_files import list2file
from src.grammar_learner.widgets import html_table
from src.grammar_learner.pqa_table import table_rows, wide_rows
tmpath = module_path + '/tmp/'
check_dir(tmpath, True, 'none')
table = []
start = time.time()
print(UTC(), ':: module_path =', module_path)

2018-11-26 10:53:04 UTC :: module_path = /home/obaskov/94/language-learning


## Corpus test settings

In [2]:
out_dir = module_path + '/output/Grammar-Rules-Generalization-' + str(UTC())[:10]
corpus = 'CDS-br-text'  # 'CDS-caps-br-text'  # 2018-11-22 shorter names
dataset = 'LG-E-clean'  # 2018-10-29: only 100% parsed, shorter names: 
lines = [[0, corpus, dataset, 0, 0, 'none'], 
         [1, corpus, dataset, 0, 0, 'rules'],
         [2, corpus, dataset, 0, 0, 'updated'],
         [3, corpus, dataset, 0, 0, 'new']] 
rp = module_path + '/data/CDS-br-text/LG-E-clean'  # "clean-clean" renamed
cp = rp  # corpus path = reference_path :: use 'gold' parses as test corpus
runs = (1,1)
kwargs = {
    'left_wall'     :   ''          ,
    'period'        :   False       ,
    'context'       :   2           ,
    'word_space'    :   'sparse'    ,
    'clustering'    :   ['agglomerative', 'ward', 'euclidean'],
    'cluster_range' :   400         ,
    'clustering_metric' : ['silhouette', 'cosine'],
    'grammar_rules' :   2           ,
    'rules_merge'       :   0.8     , # grammar rules merge threshold
    'rules_aggregation' :   0.2     , # grammar rules aggregation threshold
    'top_level'         :   0.1     , # top-level rules generalization threshold
    'tmpath'        :   tmpath      , 
    'verbose'       :   'min'       ,
    'template_path' :   'poc-turtle',
    'linkage_limit' :   1000        }

## 1 line test

In [3]:
%%capture
kwargs['clustering'] = ['agglomerative', 'ward', 'euclidean']
kwargs['cluster_range'] = 400
kwargs['rules_aggregation'] = 0.1  # default 0.2
#a0, _, header, log0, rules0 = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)
a0, _, header, log0, rules0 = wide_rows([lines[0], lines[3]], out_dir, cp, rp, runs, **kwargs)
display(html_table([header] + a0))

In [4]:
display(html_table([header] + a0))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
0,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[359, 30, 25, 14, 11]"
3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.1,198,---,0.0,99%,89%,0.90,"[364, 359, 30, 11, 9]"


# 3 versions of gramar rules generalization:
3 levels of generalization:  
1. Top-level hierarchy of "abstract" categories joining multipble grammar rules (categories)  
2. Grammar categories forming Ling Grammar rules, indexed AA...ZZ
3. Sub-categories of grammar categories.  

Column "G12n" (Generalization) describes levels 2,3 agglomeration:
- none -- no generalization,  
- rules -- legacy version of jaccard index based generalization (~June 2018),  
- updated -- enhanced hierarchical generalization,  
- new -- fast iterative generalization providing almost flat sub-category level.

## Linkage = "ward", affinity = "euclidean" (the only choice)

In [5]:
%%capture
kwargs['clustering'] = ['agglomerative', 'ward', 'euclidean']
kwargs['rules_aggregation'] = 0.1  # default 0.2
a1, _, header, log1, rules1 = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)

In [6]:
display(html_table([header] + a1))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
0,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[359, 30, 25, 14, 11]"
1,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,rules,0.1,249,---,0.0,99%,98%,0.98,"[359, 51, 30, 28, 27]"
2,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,updated,0.1,215,---,0.0,99%,97%,0.98,"[359, 48, 39, 30, 25]"
3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.1,198,---,0.0,99%,89%,0.90,"[364, 359, 30, 11, 9]"


## "Complete" linkage, "manhattan" affinity

In [7]:
%%capture
kwargs['clustering'] = ['agglomerative', 'complete', 'manhattan']
a2, _, header, log2, rules2 = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)

In [8]:
display(html_table([header] + a2))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
0,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
1,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,rules,0.1,252,---,0.0,99%,97%,0.98,"[466, 37, 34, 23, 17]"
2,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,updated,0.1,218,---,0.0,99%,97%,0.98,"[466, 37, 33, 31, 23]"
3,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,new,0.1,194,---,0.0,99%,89%,0.90,"[466, 283, 37, 11, 9]"


## "Complete" linkage, "cosine" affinity

In [9]:
%%capture
kwargs['rules_aggregation'] = 0.1
kwargs['clustering'] = ['agglomerative', 'complete', 'cosine']
a3, _, header, log3, rules3 = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)

In [10]:
display(html_table([header] + a3))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
0,CDS-br-text,LG-E-clean,dALCCd,complete,cosine,none,---,400,---,0.0,99%,89%,0.90,"[124, 53, 43, 21, 14]"
1,CDS-br-text,LG-E-clean,dALCCd,complete,cosine,rules,0.1,244,---,0.0,99%,85%,0.86,"[338, 124, 21, 17, 14]"
2,CDS-br-text,LG-E-clean,dALCCd,complete,cosine,updated,0.1,207,---,0.0,99%,83%,0.84,"[338, 124, 67, 26, 20]"
3,CDS-br-text,LG-E-clean,dALCCd,complete,cosine,new,0.1,370,---,0.0,99%,89%,0.90,"[124, 59, 53, 29, 21]"


## "Average" linkage, "cosine" affinity -- similarities less 0.1

In [11]:
%%capture
kwargs['rules_aggregation'] = 0.05   # no generalization wiht 0.1 similarity!
kwargs['clustering'] = ['agglomerative', 'average', 'cosine']
a4, _, header, log4, rules4 = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)

In [12]:
display(html_table([header] + a4))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
0,CDS-br-text,LG-E-clean,dALACd,average,cosine,none,---,400,---,0.0,99%,97%,0.98,"[98, 95, 25, 24, 17]"
1,CDS-br-text,LG-E-clean,dALACd,average,cosine,rules,0.05,291,---,0.0,99%,83%,0.84,"[476, 54, 26, 15, 12]"
2,CDS-br-text,LG-E-clean,dALACd,average,cosine,updated,0.05,283,---,0.0,99%,83%,0.84,"[476, 56, 26, 21, 15]"
3,CDS-br-text,LG-E-clean,dALACd,average,cosine,new,0.05,359,---,0.0,99%,89%,0.89,"[315, 98, 58, 24, 13]"


# Varying rules aggregation parameters

## Rules generalization merge threshold = 0.1

In [13]:
%%capture
kwargs['rules_aggregation'] = 0.1
t1 = []
n = 0
for linkage in ['ward', 'complete', 'average']:
    n += 1
    m = 0
    for affinity in ['euclidean', 'manhattan', 'cosine']:
        if linkage == 'ward' and affinity != 'euclidean': continue
        # m += 1
        # lines[0][0] = round(n + 0.1*m, 1)
        lines[0][0] = ''
        m += 1
        lines[1][0] = round(n + 0.1*m, 1)
        m += 1
        lines[2][0] = round(n + 0.1*m, 1)
        m += 1
        lines[3][0] = round(n + 0.1*m, 1)
        kwargs['clustering'] = ['agglomerative', linkage, affinity]
        a, _, header, log, _ = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)
        t1.extend(a)
        table.extend(a)

In [14]:
display(html_table([header] + t1))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[359, 30, 25, 14, 11]"
1.1,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,rules,0.1,249,---,0.0,99%,98%,0.98,"[359, 51, 30, 28, 27]"
1.2,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,updated,0.1,215,---,0.0,99%,97%,0.98,"[359, 48, 39, 30, 25]"
1.3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.1,198,---,0.0,99%,89%,0.90,"[364, 359, 30, 11, 9]"
,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
2.1,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,rules,0.1,252,---,0.0,99%,97%,0.98,"[466, 37, 34, 23, 17]"
2.2,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,updated,0.1,218,---,0.0,99%,97%,0.98,"[466, 37, 33, 31, 23]"
2.3,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,new,0.1,194,---,0.0,99%,89%,0.90,"[466, 283, 37, 11, 9]"
,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
2.4,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,rules,0.1,252,---,0.0,99%,97%,0.98,"[466, 37, 34, 23, 17]"


## Rules generalization merge threshold = 0.05

In [15]:
%%capture
kwargs['rules_aggregation'] = 0.05
t2 = []
for linkage in ['ward', 'complete', 'average']:
    n += 1
    m = 0
    for affinity in ['euclidean', 'manhattan', 'cosine']:
        if linkage == 'ward' and affinity != 'euclidean': continue
        # m += 1
        # lines[0][0] = round(n + 0.1*m, 1)
        lines[0][0] = ''
        m += 1
        lines[1][0] = round(n + 0.1*m, 1)
        m += 1
        lines[2][0] = round(n + 0.1*m, 1)
        m += 1
        lines[3][0] = round(n + 0.1*m, 1)
        kwargs['clustering'] = ['agglomerative', linkage, affinity]
        a, _, header, log, _ = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)
        t2.extend(a)
        table.extend(a)

In [16]:
display(html_table([header] + t2))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[359, 30, 25, 14, 11]"
4.1,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,rules,0.05,149,---,0.0,99%,90%,0.91,"[413, 193, 32, 30, 26]"
4.2,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,updated,0.05,68,---,0.0,99%,80%,0.81,"[672, 174, 21, 19, 14]"
4.3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.05,81,---,0.0,99%,76%,0.77,"[585, 359, 6, 2, 1]"
,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
5.1,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,rules,0.05,146,---,0.0,99%,90%,0.90,"[549, 117, 68, 37, 13]"
5.2,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,updated,0.05,46,---,0.0,99%,76%,0.77,"[716, 245, 10, 5, 4]"
5.3,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,new,0.05,85,---,0.0,99%,77%,0.78,"[466, 438, 37, 6, 2]"
,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
5.4,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,rules,0.05,146,---,0.0,99%,90%,0.90,"[549, 117, 68, 37, 13]"


## Start with 300 clusters, merge threshold = 0.05

In [17]:
%%capture
kwargs['cluster_range'] = 300
kwargs['rules_aggregation'] = 0.05
t3 = []
for linkage in ['ward', 'complete', 'average']:
    n += 1
    m = 0
    for affinity in ['euclidean', 'manhattan', 'cosine']:
        if linkage == 'ward' and affinity != 'euclidean': continue
        lines[0][0] = ''
        m += 1
        lines[1][0] = round(n + 0.1*m, 1)
        m += 1
        lines[2][0] = round(n + 0.1*m, 1)
        m += 1
        lines[3][0] = round(n + 0.1*m, 1)
        kwargs['clustering'] = ['agglomerative', linkage, affinity]
        a, _, header, log, _ = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)
        t3.extend(a)
        table.extend(a)

In [18]:
display(html_table([header] + t3))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,300,---,0.0,99%,97%,0.98,"[437, 34, 29, 17, 15]"
7.1,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,rules,0.05,107,---,0.0,99%,83%,0.83,"[748, 35, 22, 17, 11]"
7.2,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,updated,0.05,36,---,0.0,99%,74%,0.75,"[961, 16, 9, 5, 4]"
7.3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.05,73,---,0.0,99%,78%,0.79,"[514, 437, 8, 2, 1]"
,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,none,---,300,---,0.0,99%,96%,0.97,"[607, 23, 12, 11, 10]"
8.1,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,rules,0.05,112,---,0.0,99%,87%,0.88,"[742, 32, 25, 20, 19]"
8.2,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,updated,0.05,19,---,0.0,99%,72%,0.73,"[1014, 2, 1, 0]"
8.3,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,new,0.05,76,---,0.0,99%,79%,0.80,"[607, 332, 8, 7, 3]"
,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,none,---,300,---,0.0,99%,96%,0.97,"[607, 23, 12, 11, 10]"
8.4,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,rules,0.05,112,---,0.0,99%,87%,0.88,"[742, 32, 25, 20, 19]"


## Start with 200 clusters, merge threshold = 0.1

In [19]:
%%capture
kwargs['cluster_range'] = 200
kwargs['rules_aggregation'] = 0.1
t4 = []
for linkage in ['ward', 'complete', 'average']:
    n += 1
    m = 0
    for affinity in ['euclidean', 'manhattan', 'cosine']:
        if linkage == 'ward' and affinity != 'euclidean': continue
        lines[0][0] = ''
        m += 1
        lines[1][0] = round(n + 0.1*m, 1)
        m += 1
        lines[2][0] = round(n + 0.1*m, 1)
        m += 1
        lines[3][0] = round(n + 0.1*m, 1)
        kwargs['clustering'] = ['agglomerative', linkage, affinity]
        a, _, header, log, _ = wide_rows(lines, out_dir, cp, rp, runs, **kwargs)
        t4.extend(a)
        table.extend(a)

In [20]:
display(html_table([header] + t4))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,200,---,0.0,99%,98%,0.98,"[569, 34, 32, 26, 20]"
10.1,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,rules,0.1,128,---,0.0,99%,96%,0.96,"[569, 38, 34, 32, 27]"
10.2,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,updated,0.1,108,---,0.0,99%,95%,0.95,"[569, 104, 57, 34, 32]"
10.3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.1,130,---,0.0,99%,96%,0.96,"[569, 66, 34, 32, 26]"
,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,none,---,200,---,0.0,99%,96%,0.97,"[817, 4, 3, 2, 1]"
11.1,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,rules,0.1,124,---,0.0,99%,93%,0.93,"[817, 38, 17, 7, 6]"
11.2,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,updated,0.1,102,---,0.0,99%,91%,0.92,"[817, 58, 36, 6, 5]"
11.3,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,new,0.1,131,---,0.0,99%,92%,0.93,"[817, 33, 24, 11, 4]"
,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,none,---,200,---,0.0,99%,96%,0.97,"[817, 4, 3, 2, 1]"
11.4,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,rules,0.1,124,---,0.0,99%,93%,0.93,"[817, 38, 17, 7, 6]"


# All tests

In [21]:
display(html_table([header] + table))

Line,Corpus,Parsing,Space,Linkage,Affinity,G12n,Threshold,Rules,NN,SI,PA,PQ,F1,Top 5 cluster sizes
,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[359, 30, 25, 14, 11]"
1.1,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,rules,0.1,249,---,0.0,99%,98%,0.98,"[359, 51, 30, 28, 27]"
1.2,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,updated,0.1,215,---,0.0,99%,97%,0.98,"[359, 48, 39, 30, 25]"
1.3,CDS-br-text,LG-E-clean,dALWEd,ward,euclidean,new,0.1,198,---,0.0,99%,89%,0.90,"[364, 359, 30, 11, 9]"
,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
2.1,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,rules,0.1,252,---,0.0,99%,97%,0.98,"[466, 37, 34, 23, 17]"
2.2,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,updated,0.1,218,---,0.0,99%,97%,0.98,"[466, 37, 33, 31, 23]"
2.3,CDS-br-text,LG-E-clean,dALCEd,complete,euclidean,new,0.1,194,---,0.0,99%,89%,0.90,"[466, 283, 37, 11, 9]"
,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,none,---,400,---,0.0,99%,96%,0.97,"[466, 37, 11, 9, 8]"
2.4,CDS-br-text,LG-E-clean,dALCMd,complete,manhattan,rules,0.1,252,---,0.0,99%,97%,0.98,"[466, 37, 34, 23, 17]"


In [22]:
print(UTC(), ':: finished, elapsed', str(round((time.time()-start)/3600.0, 1)), 'hours')
table_str = list2file(table, out_dir + '/table.txt')
print('Results saved to', out_dir + '/table.txt')

2018-11-26 11:36:11 UTC :: finished, elapsed 0.7 hours
Results saved to /home/obaskov/94/language-learning/output/Grammar-Rules-Generalization-2018-11-26/table.txt
